### Query Government Census API for US Census Broadband Data (ACS 5 year estimate, 2022)

In [ ]:
import requests
import pandas as pd
import time
from pathlib import Path

# Output path (change if needed)
output_path = Path("../data/census_broadband.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

# Census variables to query
variables = [
    "NAME",              # County name
    "B28002_001E",       # Total households
    "B28002_004E",       # HH with broadband
    "B28002_013E",       # HH with no internet
    "B28003_001E",       # HH with computer
    "B28003_004E",       # HH with computer and broadband
    "B01003_001E",       # Total population
    "B19013_001E"        # Median household income
]

# Mapping from FIPS to state abbreviations
fips_to_state = {
    "01": "AL", "02": "AK", "04": "AZ", "05": "AR", "06": "CA", "08": "CO",
    "09": "CT", "10": "DE", "11": "DC", "12": "FL", "13": "GA", "15": "HI",
    "16": "ID", "17": "IL", "18": "IN", "19": "IA", "20": "KS", "21": "KY",
    "22": "LA", "23": "ME", "24": "MD", "25": "MA", "26": "MI", "27": "MN",
    "28": "MS", "29": "MO", "30": "MT", "31": "NE", "32": "NV", "33": "NH",
    "34": "NJ", "35": "NM", "36": "NY", "37": "NC", "38": "ND", "39": "OH",
    "40": "OK", "41": "OR", "42": "PA", "44": "RI", "45": "SC", "46": "SD",
    "47": "TN", "48": "TX", "49": "UT", "50": "VT", "51": "VA", "53": "WA",
    "54": "WV", "55": "WI", "56": "WY"
}

# Base URL for ACS 5-Year 2022
base_url = "https://api.census.gov/data/2022/acs/acs5"

# All valid state FIPS codes (01–56, excluding deprecated)
state_fips = [
    f"{i:02d}" for i in range(1, 57)
    if i not in {3, 7, 14, 43, 52}
]

# Start fetching
all_data = []
print("📡 Starting data fetch from Census API...")

for fips in state_fips:
    print(f"🔍 Querying counties in state FIPS {fips} ({fips_to_state.get(fips, 'Unknown')})...")
    params = {
        "get": ",".join(variables),
        "for": "county:*",
        "in": f"state:{fips}"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        columns = data[0]
        rows = data[1:]
        df = pd.DataFrame(rows, columns=columns)
        df["state_abbr"] = fips_to_state.get(fips, "Unknown")
        all_data.append(df)
        print(f"Retrieved {len(df)} records.")
    else:
        print(f"Failed for state {fips}: status code {response.status_code}")
    time.sleep(0.5)  # throttle requests

# Combine and save
final_df = pd.concat(all_data, ignore_index=True)
final_df.to_csv(output_path, index=False)
print(f"Data saved to: {output_path.resolve()}")

📡 Starting data fetch from Census API...
🔍 Querying counties in state FIPS 01 (AL)...
  ✅ Retrieved 67 records.
🔍 Querying counties in state FIPS 02 (AK)...
  ✅ Retrieved 30 records.
🔍 Querying counties in state FIPS 04 (AZ)...
  ✅ Retrieved 15 records.
🔍 Querying counties in state FIPS 05 (AR)...
  ✅ Retrieved 75 records.
🔍 Querying counties in state FIPS 06 (CA)...
  ✅ Retrieved 58 records.
🔍 Querying counties in state FIPS 08 (CO)...
  ✅ Retrieved 64 records.
🔍 Querying counties in state FIPS 09 (CT)...
  ✅ Retrieved 9 records.
🔍 Querying counties in state FIPS 10 (DE)...
  ✅ Retrieved 3 records.
🔍 Querying counties in state FIPS 11 (DC)...
  ✅ Retrieved 1 records.
🔍 Querying counties in state FIPS 12 (FL)...
  ✅ Retrieved 67 records.
🔍 Querying counties in state FIPS 13 (GA)...
  ✅ Retrieved 159 records.
🔍 Querying counties in state FIPS 15 (HI)...
  ✅ Retrieved 5 records.
🔍 Querying counties in state FIPS 16 (ID)...
  ✅ Retrieved 44 records.
🔍 Querying counties in state FIPS 17 (I

### Change Variable Names After Data Retrieval

In [3]:
import pandas as pd

# Load data
df = pd.read_csv("../data/census_broadband.csv")

# Rename columns
df = df.rename(columns={
    "NAME": "county_name",
    "B28002_001E": "total_households",
    "B28002_004E": "households_with_broadband",
    "B28002_013E": "households_with_no_internet",
    "B28003_001E": "households_with_computer",
    "B28003_004E": "households_with_computer_and_broadband",
    "B01003_001E": "total_population",
    "B19013_001E": "median_household_income",
    "state": "state_fips",
    "county": "county_fips"
})

# Save cleaned version (optional)
df.to_csv("../data/census_broadband_cleaned.csv", index=False)

# Preview
print(df.head())

               county_name  total_households  households_with_broadband  \
0  Autauga County, Alabama             22308                      19869   
1  Baldwin County, Alabama             90802                      80742   
2  Barbour County, Alabama              9016                       6151   
3     Bibb County, Alabama              7216                       5752   
4   Blount County, Alabama             21626                      17890   

   households_with_no_internet  households_with_computer  \
0                         2111                     22308   
1                         8127                     90802   
2                         2194                      9016   
3                         1391                      7216   
4                         3243                     21626   

   households_with_computer_and_broadband  total_population  \
0                                   19744             58761   
1                                   80169            233420   

### Clean Our Data Files gathered from [https://www.fcc.gov/form-477-county-data-internet-access-services]

In [5]:
import pandas as pd
from pathlib import Path

# Load ACS broadband data
acs_path = Path("../data/census_broadband_cleaned.csv")
acs_df = pd.read_csv(acs_path, dtype={"state_fips": str, "county_fips": str})

# Load FCC tier data (decode using ISO-8859-1 due to special characters)
fcc_path = Path("../data/county_tiers_201406_202312.csv")
fcc_df = pd.read_csv(fcc_path, encoding="ISO-8859-1")

# Filter FCC data to December 2023
fcc_df = fcc_df[(fcc_df["Year"] == 2023) & (fcc_df["Month"] == 12)]

# Rename FCC columns for clarity
fcc_df = fcc_df.rename(columns={
    "FIPS": "fips_code",
    "Housing_Units": "housing_units",
    "Tier_1": "connections_200kbps_per_1000",
    "Tier_2": "connections_10mbps_per_1000",
    "Tier_3": "connections_25mbps_per_1000",
    "Tier_4": "connections_100mbps_per_1000"
})

# Pad FIPS codes
fcc_df["fips_code"] = fcc_df["fips_code"].astype(str).str.zfill(5)
acs_df["fips_code"] = acs_df["state_fips"].str.zfill(2) + acs_df["county_fips"].str.zfill(3)

# Merge ACS and FCC data on fips_code
merged_df = pd.merge(acs_df, fcc_df, on="fips_code", how="left")

# Save merged dataset
output_path = Path("../data/merged_broadband_data.csv")
merged_df.to_csv(output_path, index=False)

print(f"Merged dataset saved to: {output_path.resolve()}")
print(f"Final dataset shape: {merged_df.shape}")


Merged dataset saved to: /home/platinumfish/Desktop/Coding_projects/data_visualization_project_2025/data/merged_broadband_data.csv
Final dataset shape: (3144, 23)


### Merge both @ and @

In [7]:
import pandas as pd
from pathlib import Path

# === Load ACS Data ===
acs_path = Path("../data/census_broadband_cleaned.csv")
acs_df = pd.read_csv(acs_path, dtype={"state_fips": str, "county_fips": str})
acs_df["fips_code"] = acs_df["state_fips"].str.zfill(2) + acs_df["county_fips"].str.zfill(3)

# === Load FCC Connections Data ===
fcc_path = Path("../data/county_connections_200906_202312.csv")
fcc_df = pd.read_csv(fcc_path, encoding="ISO-8859-1")

# Filter to December 2023
fcc_df = fcc_df[(fcc_df["year"] == 2023) & (fcc_df["month"] == 12)]

# Rename columns
fcc_df = fcc_df.rename(columns={
    "countycode": "fips_code",
    "statename": "state_name",
    "countyname": "county_name",
    "consumer": "residential_connections_k",
    "non_consumer": "nonresidential_connections_k",
    "all": "total_connections_k"
})

# Pad FIPS to 5 digits
fcc_df["fips_code"] = fcc_df["fips_code"].astype(str).str.zfill(5)

# === Merge on fips_code ===
merged_df = pd.merge(acs_df, fcc_df, on="fips_code", how="left")

# === Save output ===
output_path = Path("../data/merged_broadband_connections.csv")
merged_df.to_csv(output_path, index=False)

print(f"Merged dataset saved to: {output_path.resolve()}")
print(f"Final shape: {merged_df.shape}")


Merged dataset saved to: /home/platinumfish/Desktop/Coding_projects/data_visualization_project_2025/data/merged_broadband_connections.csv
Final shape: (3144, 19)


### Combine the @ Data to create a Comprehensive Dataset

In [9]:
import pandas as pd
from pathlib import Path

# === Load ACS Data ===
acs_path = Path("../data/census_broadband_cleaned.csv")
acs_df = pd.read_csv(acs_path, dtype={"state_fips": str, "county_fips": str})
acs_df["fips_code"] = acs_df["state_fips"].str.zfill(2) + acs_df["county_fips"].str.zfill(3)

# === Load FCC Tier Data ===
tier_path = Path("../data/county_tiers_201406_202312.csv")
tier_df = pd.read_csv(tier_path, encoding="ISO-8859-1")
tier_df = tier_df[(tier_df["Year"] == 2023) & (tier_df["Month"] == 12)]
tier_df = tier_df.rename(columns={
    "FIPS": "fips_code",
    "Housing_Units": "housing_units",
    "Tier_1": "connections_200kbps_per_1000",
    "Tier_2": "connections_10mbps_per_1000",
    "Tier_3": "connections_25mbps_per_1000",
    "Tier_4": "connections_100mbps_per_1000"
})
tier_df["fips_code"] = tier_df["fips_code"].astype(str).str.zfill(5)

# === Load FCC Connection Volume Data ===
conn_path = Path("../data/county_connections_200906_202312.csv")
conn_df = pd.read_csv(conn_path, encoding="ISO-8859-1")
conn_df = conn_df[(conn_df["year"] == 2023) & (conn_df["month"] == 12)]
conn_df = conn_df.rename(columns={
    "countycode": "fips_code",
    "statename": "state_name",
    "countyname": "county_name",
    "consumer": "residential_connections_k",
    "non_consumer": "nonresidential_connections_k",
    "all": "total_connections_k"
})
conn_df["fips_code"] = conn_df["fips_code"].astype(str).str.zfill(5)

# === Merge all datasets ===
merged = pd.merge(acs_df, tier_df, on="fips_code", how="left")
merged = pd.merge(merged, conn_df[[
    "fips_code", "year", "month",
    "residential_connections_k", "nonresidential_connections_k", "total_connections_k"
]], on="fips_code", how="left")

# === Reorder columns ===
column_order = [
    # Location + time
    "fips_code", "state_fips", "county_fips", "state_abbr", "county_name", "year", "month",

    # Demographics
    "total_population", "median_household_income",

    # ACS: access + devices
    "total_households", "households_with_computer", "households_with_computer_and_broadband",
    "households_with_broadband", "households_with_no_internet",

    # FCC: availability per 1,000 units
    "housing_units",
    "connections_200kbps_per_1000", "connections_10mbps_per_1000",
    "connections_25mbps_per_1000", "connections_100mbps_per_1000",

    # FCC: actual connections (volume)
    "residential_connections_k", "nonresidential_connections_k", "total_connections_k"
]

# Keep only columns that exist in the merged dataframe
final_columns = [col for col in column_order if col in merged.columns]
merged = merged[final_columns]

# === Save final dataset ===
output_path = Path("../data/merged_broadband_full.csv")
merged.to_csv(output_path, index=False)

print(f"Merged dataset saved to: {output_path.resolve()}")
print(f"Final shape: {merged.shape}")

Merged dataset saved to: /home/platinumfish/Desktop/Coding_projects/data_visualization_project_2025/data/merged_broadband_full.csv
Final shape: (3144, 22)


### Create Additional Variables Based on Actual Broadband Usage

In [12]:
import pandas as pd
from pathlib import Path

# Load the merged dataset
merged_df = pd.read_csv("../data/merged_broadband_full.csv")

# Calculate broadband access rate
merged_df["broadband_access_rate"] = (
    merged_df["households_with_broadband"] / merged_df["total_households"]
).round(2)

# Calculate residential connections per household
merged_df["res_conn_per_household"] = (
    merged_df["residential_connections_k"] * 1000 / merged_df["total_households"]
).round(2)

# Save updated dataset
output_path = Path("../data/merged_broadband_full.csv")
merged_df.to_csv(output_path, index=False)

print(f"Merged dataset saved to: {output_path.resolve()}")

Merged dataset saved to: /home/platinumfish/Desktop/Coding_projects/data_visualization_project_2025/data/merged_broadband_full.csv


In [19]:
import requests
import pandas as pd
from pathlib import Path
import time

# === Clear memory from previous runs ===
all_data = []
edu_df = None

# === File paths ===
acs_path = Path("../data/merged_broadband_full.csv")
edu_output_path = Path("../data/merged_broadband_full.csv")

# === Load existing dataset ===
df_main = pd.read_csv(acs_path, dtype={"state_fips": str, "county_fips": str})
df_main["fips_code"] = df_main["state_fips"].str.zfill(2) + df_main["county_fips"].str.zfill(3)

# === Education variables from ACS B15003 ===
variables = [
    "NAME",
    "B15003_001E",  # Total population 25+
    "B15003_017E",  # High school graduate
    "B15003_022E",  # Bachelor's degree
    "B15003_025E"   # Graduate/professional degree
]

base_url = "https://api.census.gov/data/2022/acs/acs5"
state_fips = [f"{i:02d}" for i in range(1, 57) if i not in {3, 7, 14, 43, 52}]

# === Robust request function with retries ===
def fetch_with_retries(url, params, max_retries=8):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params, timeout=10)
            if response.status_code == 200:
                return response
            else:
                print(f"  Request failed with status {response.status_code}, retrying in {2 ** attempt} seconds...")
        except requests.exceptions.RequestException as e:
            print(f"  Error: {e}, retrying in {2 ** attempt} seconds...")
        time.sleep(2 ** attempt)
    print("  Max retries exceeded. Skipping this state.")
    return None

# === Begin pulling ===
print("Starting education data fetch from Census API...")

for fips in state_fips:
    print(f"🔍 Querying counties in state FIPS {fips}...")
    params = {
        "get": ",".join(variables),
        "for": "county:*",
        "in": f"state:{fips}"
    }
    response = fetch_with_retries(base_url, params=params)
    if response:
        try:
            rows = response.json()
            headers = rows[0]
            data = rows[1:]
            df = pd.DataFrame(data, columns=headers)
            df["state"] = fips
            df["fips_code"] = df["state"].str.zfill(2) + df["county"].str.zfill(3)
            all_data.append(df)
            print(f"Retrieved {len(df)} records.")
        except Exception as e:
            print(f"  Failed to parse response for state {fips}: {e}")
    time.sleep(2)

# === Combine data ===
edu_df = pd.concat(all_data, ignore_index=True)
print(f"Total counties pulled: {len(edu_df)}")

# === Convert columns ===
for col in ["B15003_001E", "B15003_017E", "B15003_022E", "B15003_025E"]:
    edu_df[col] = pd.to_numeric(edu_df[col], errors="coerce")

# === Compute percentages ===
edu_df["pct_hs_or_higher"] = (
    (edu_df["B15003_017E"] + edu_df["B15003_022E"] + edu_df["B15003_025E"]) / edu_df["B15003_001E"]
).round(2)

edu_df["pct_ba_or_higher"] = (
    (edu_df["B15003_022E"] + edu_df["B15003_025E"]) / edu_df["B15003_001E"]
).round(2)

# === Trim and merge ===
edu_df = edu_df[["fips_code", "pct_hs_or_higher", "pct_ba_or_higher"]]
df_merged = pd.merge(df_main, edu_df, on="fips_code", how="left")

# === Save ===
df_merged.to_csv(edu_output_path, index=False)
print(f"Updated dataset with education variables saved to: {edu_output_path.resolve()}")

📡 Starting education data fetch from Census API...
🔍 Querying counties in state FIPS 01...
  ✅ Retrieved 67 records.
🔍 Querying counties in state FIPS 02...
  Error: HTTPSConnectionPool(host='api.census.gov', port=443): Read timed out. (read timeout=10), retrying in 1 seconds...
  Error: HTTPSConnectionPool(host='api.census.gov', port=443): Read timed out. (read timeout=10), retrying in 2 seconds...
  ✅ Retrieved 30 records.
🔍 Querying counties in state FIPS 04...
  ✅ Retrieved 15 records.
🔍 Querying counties in state FIPS 05...
  ✅ Retrieved 75 records.
🔍 Querying counties in state FIPS 06...
  ✅ Retrieved 58 records.
🔍 Querying counties in state FIPS 08...
  ✅ Retrieved 64 records.
🔍 Querying counties in state FIPS 09...
  ✅ Retrieved 9 records.
🔍 Querying counties in state FIPS 10...
  ✅ Retrieved 3 records.
🔍 Querying counties in state FIPS 11...
  ✅ Retrieved 1 records.
🔍 Querying counties in state FIPS 12...
  ✅ Retrieved 67 records.
🔍 Querying counties in state FIPS 13...
  ✅ Re

### Gather ACS-5 Year Racial Identity Data

In [18]:
import requests
import pandas as pd
from pathlib import Path
import time

# === File paths ===
acs_path = Path("../data/merged_broadband_full.csv")
output_path = Path("../data/merged_broadband_full.csv")

# === Load existing dataset ===
df_main = pd.read_csv(acs_path, dtype={"state_fips": str, "county_fips": str})
df_main["fips_code"] = df_main["state_fips"].str.zfill(2) + df_main["county_fips"].str.zfill(3)

# === Census variables to pull ===
variables = [
    "NAME",
    "B02001_001E",  # Total population
    "B02001_002E",  # White alone
    "B02001_003E",  # Black or African American alone
    "B02001_004E",  # American Indian and Alaska Native alone
    "B02001_005E",  # Asian alone
    "B02001_007E",  # Some other race alone
    "B02001_008E",  # Two or more races
    "B03003_003E"   # Hispanic or Latino
]

base_url = "https://api.census.gov/data/2022/acs/acs5"
state_fips = [f"{i:02d}" for i in range(1, 57) if i not in {3, 7, 14, 43, 52}]
all_data = []

# === Retryable fetch ===
def fetch_with_retries(url, params, max_retries=5):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params, timeout=10)
            if response.status_code == 200:
                return response
            print(f"  Request failed with status {response.status_code}, retrying...")
        except requests.exceptions.RequestException as e:
            print(f"  Error: {e}, retrying in {2 ** attempt} seconds...")
            time.sleep(2 ** attempt)
    print("  Max retries exceeded. Skipping this state.")
    return None

# === Begin pull ===
print("Starting detailed race and ethnicity data pull...")

for fips in state_fips:
    print(f"Querying counties in state FIPS {fips}...")
    params = {
        "get": ",".join(variables),
        "for": "county:*",
        "in": f"state:{fips}"
    }
    response = fetch_with_retries(base_url, params=params)
    if response:
        try:
            rows = response.json()
            headers = rows[0]
            data = rows[1:]
            df = pd.DataFrame(data, columns=headers)
            df["state"] = fips
            df["fips_code"] = df["state"].str.zfill(2) + df["county"].str.zfill(3)
            all_data.append(df)
            print(f"  Retrieved {len(df)} records.")
        except Exception as e:
            print(f"  Failed to parse response for state {fips}: {e}")
    time.sleep(2)

# === Combine and convert ===
race_df = pd.concat(all_data, ignore_index=True)
print(f"Total counties pulled: {len(race_df)}")

# Convert numeric columns
cols = [
    "B02001_001E", "B02001_002E", "B02001_003E",
    "B02001_004E", "B02001_005E", "B02001_007E",
    "B02001_008E", "B03003_003E"
]
race_df[cols] = race_df[cols].apply(pd.to_numeric, errors="coerce")

# === Calculate percent composition ===
race_df["percent_white_alone"] = (race_df["B02001_002E"] / race_df["B02001_001E"]).round(2)
race_df["percent_black_alone"] = (race_df["B02001_003E"] / race_df["B02001_001E"]).round(2)
race_df["percent_native_american_alone"] = (race_df["B02001_004E"] / race_df["B02001_001E"]).round(2)
race_df["percent_asian_alone"] = (race_df["B02001_005E"] / race_df["B02001_001E"]).round(2)
race_df["percent_other_race_alone"] = (race_df["B02001_007E"] / race_df["B02001_001E"]).round(2)
race_df["percent_two_or_more_races"] = (race_df["B02001_008E"] / race_df["B02001_001E"]).round(2)
race_df["percent_hispanic_latino"] = (race_df["B03003_003E"] / race_df["B02001_001E"]).round(2)

# Keep only relevant columns
race_df = race_df[[
    "fips_code",
    "percent_white_alone",
    "percent_black_alone",
    "percent_native_american_alone",
    "percent_asian_alone",
    "percent_other_race_alone",
    "percent_two_or_more_races",
    "percent_hispanic_latino"
]]

# === Merge and save ===
df_merged = pd.merge(df_main, race_df, on="fips_code", how="left")
df_merged.to_csv(output_path, index=False)

print(f"Updated dataset with detailed race and ethnicity variables saved to: {output_path.resolve()}")

Starting detailed race and ethnicity data pull...
Querying counties in state FIPS 01...
  Error: HTTPSConnectionPool(host='api.census.gov', port=443): Read timed out. (read timeout=10), retrying in 1 seconds...
  Request failed with status 503, retrying...
  Error: HTTPSConnectionPool(host='api.census.gov', port=443): Read timed out. (read timeout=10), retrying in 4 seconds...
  Retrieved 67 records.
Querying counties in state FIPS 02...
  Error: HTTPSConnectionPool(host='api.census.gov', port=443): Read timed out. (read timeout=10), retrying in 1 seconds...
  Retrieved 30 records.
Querying counties in state FIPS 04...
  Error: HTTPSConnectionPool(host='api.census.gov', port=443): Read timed out. (read timeout=10), retrying in 1 seconds...
  Retrieved 15 records.
Querying counties in state FIPS 05...
  Error: HTTPSConnectionPool(host='api.census.gov', port=443): Read timed out. (read timeout=10), retrying in 1 seconds...
  Error: HTTPSConnectionPool(host='api.census.gov', port=443): Re

### Remove the Alaskan Aleutian Islands to Prevent Visualization Issues, also Include Geographic Data for Graphing

In [22]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

# === File paths ===
csv_path = Path("../data/merged_broadband_full.csv")
shp_path = Path("../data/cb_2023_us_county_500k/cb_2023_us_county_500k.shp")

# === Load broadband dataset ===
print("Loading broadband dataset...")
df = pd.read_csv(csv_path, dtype={"state_fips": str, "county_fips": str})
df["fips_code"] = df["state_fips"].str.zfill(2) + df["county_fips"].str.zfill(3)
print(f"  Loaded {len(df)} broadband records")

# === Load shapefile ===
print("Loading county shapefile...")
gdf = gpd.read_file(shp_path)
gdf["fips_code"] = gdf["STATEFP"] + gdf["COUNTYFP"]
print(f"  Loaded {len(gdf)} geometries")

# === Merge datasets ===
print("Merging broadband data with geometries...")
merged_gdf = gdf.merge(df, on="fips_code", how="left")
print(f"  Merged shape: {merged_gdf.shape}")

# === Remove Aleutians ===
print("Removing Aleutian Islands (FIPS 02013, 02016)...")
merged_gdf = merged_gdf[~merged_gdf["fips_code"].isin(["02013", "02016"])]
print(f"Final shape after filter: {merged_gdf.shape}")

# === Save updated CSV ===
output_csv_path = Path("../data/merged_broadband_full.csv")
merged_gdf.drop(columns="geometry").to_csv(output_csv_path, index=False)
print(f"Updated dataset saved to: {output_csv_path.resolve()}")

Loading broadband dataset...
  Loaded 3144 broadband records
Loading county shapefile...
  Loaded 3235 geometries
Merging broadband data with geometries...
  Merged shape: (3235, 46)
Removing Aleutian Islands (FIPS 02013, 02016)...
Final shape after filter: (3233, 46)
Updated dataset saved to: /home/platinumfish/Desktop/Coding_projects/data_visualization_project_2025/data/merged_broadband_full.csv
